1) Install necessary packages

In [45]:
%pip install torch transformers numpy pandas scikit-learn PyPDF2 spacy nltk
%pip install transformers
%pip install transformers datasets torch scikit-learn
%pip install transformers[torch]
%pip install accelerate
%pip install torch
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu124

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cu124
Note: you may need to restart the kernel to use updated packages.


2) Loading the Data

In [46]:
import os
import PyPDF2

In [47]:
#path to pdf files
data_path = 'C:\\Users\\asha4\\OneDrive\\Desktop\\mini\\mini\data'

In [48]:
#creating list to store pdf data
pdf_as_text = []

In [49]:
for filename in os.listdir(data_path):
    if filename.endswith('.pdf'):
        file_path = os.path.join(data_path, filename)
        print(f"Attempting to open: {file_path}")

        try:
            with open(file_path, 'rb') as file:  # Open the PDF file
                pdf_reader = PyPDF2.PdfReader(file)
                text_content_of_pdf = ''

                for page in pdf_reader.pages:
                    text_content_of_pdf += page.extract_text() if page.extract_text() else ''

                pdf_as_text.append(text_content_of_pdf)

        except Exception as e:
            print(f"Failed to read {file_path}: {e}")  # This will print out why the file couldn't be opened

# Optionally print or process the text extracted from each PDF
for text in pdf_as_text:
    print(text)  # Print extracted text


Attempting to open: C:\Users\asha4\OneDrive\Desktop\mini\mini\data\15_pdf_Neubuerger_Brosch_engl.pdf
Attempting to open: C:\Users\asha4\OneDrive\Desktop\mini\mini\data\16_pdf_kip-kurz-englisch.pdf
Attempting to open: C:\Users\asha4\OneDrive\Desktop\mini\mini\data\190521_Sprachförderung_DINlang_RZ_EN_web.pdf
Attempting to open: C:\Users\asha4\OneDrive\Desktop\mini\mini\data\190927_Broschüre_Bahnstadt_final_englisch.pdf
Attempting to open: C:\Users\asha4\OneDrive\Desktop\mini\mini\data\230131_HeidelbergCARD_Flyer_2023_EN_RZ_ES (1).pdf
Attempting to open: C:\Users\asha4\OneDrive\Desktop\mini\mini\data\9781800208421-THE_NATURAL_LANGUAGE_PROCESSING_WORKSHOP.pdf
Attempting to open: C:\Users\asha4\OneDrive\Desktop\mini\mini\data\about us.pdf
Attempting to open: C:\Users\asha4\OneDrive\Desktop\mini\mini\data\acc.pdf
Attempting to open: C:\Users\asha4\OneDrive\Desktop\mini\mini\data\acc1.pdf
Attempting to open: C:\Users\asha4\OneDrive\Desktop\mini\mini\data\acc2.pdf
Attempting to open: C:\Users

3) Data Cleaning

Breakdown of the Cleaning Steps:

1. URL Removal: Extracted text might contain URLs which we remove using regular expressions.
2. Page Number Removal: Often, PDF extraction includes footer elements like page numbers which we're removing.
3. Metadata Removal: Copyright phrases and other metadata are common in PDFs.
4. Non-ASCII Characters: Remove any characters that are not standard ASCII, which helps clean up characters that did not translate well from the PDF.
5. Whitespace Normalization: Convert any sequence of whitespace characters to a single space, which helps in cleaning up the formatting.
6. Punctuation and Special Characters: Remove punctuation which is often misplaced or misinterpreted during PDF extraction.
7. Single Character Removal: Optionally, removing isolated single characters can clean up artifacts but should be used with caution as it can also remove valid data (like 'a' or 'I').
8. Case Normalization: Convert text to lowercase to standardize it and simplify further processing.
9. Trimming: Remove extra spaces at the beginning and end of the text.

In [50]:
import re
import spacy

In [51]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.2/12.8 MB 3.9 MB/s eta 0:00:04
     - -------------------------------------- 0.5/12.8 MB 5.2 MB/s eta 0:00:03
     -- ------------------------------------- 0.8/12.8 MB 5.7 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/12.8 MB 6.5 MB/s eta 0:00:02
     ----- ---------------------------------- 1.7/12.8 MB 7.2 MB/s eta 0:00:02
     ------ --------------------------------- 2.2/12.8 MB 7.9 MB/s eta 0:00:02
     -------- ------------------------------- 2.8/12.8 MB 8.5 MB/s eta 0:00:02
     ---------- ----------------------------- 3.4/12.8 MB 9.1 MB/s eta 0:00:02
     ------------ --------------------------- 4.1/12.8 MB 9.7 MB/s eta 0:00:01
     --------------- ------------------------ 5.0/12.8 MB 10.7 MB/s eta 0:00:01
     ------------------ --------------------- 5.9/12.8 MB 11.4 MB

In [52]:
nlp = spacy.load('en_core_web_sm')

In [53]:
def correct_split_hyphenated_words(text):
    text = re.sub(r'\s+-\s+', '-', text)
    return text


def nlp_handle_hyphenated_words(text):
    doc = nlp(text)
    new_text = []
    for token in doc:
        if '-' in token.text:
            if token.pos_ in ['PROPN', 'NOUN']: 
                new_text.append(token.text)
            else:
                parts = token.text.split('-')
                if len(parts) > 1:
                    sub_doc = nlp(' '.join(parts))
                    sub_parts = [sub_token.text_with_ws for sub_token in sub_doc]
                    new_text.extend(sub_parts)
                else:
                    new_text.append(token.text_with_ws)
        else:
            new_text.append(token.text_with_ws)
    return ''.join(new_text)


def preserve_hyphenated_words(text):
    hyphenated_words = re.findall(r'\w+-\w+', text)
    
    for word in hyphenated_words:
        if word.lower() == 'e-mail':
            text = text.replace(word, 'email')
    
    return text


def expand_contractions(text):
    contractions = {
        "doesn't": "does not", "isn't": "is not", "wasn't": "was not", "weren't": "were not",
        "can't": "cannot", "couldn't": "could not", "shouldn't": "should not", "wouldn't": "would not",
        "aren't": "are not", "haven't": "have not", "hasn't": "has not",
        "hadn't": "had not", "won't": "will not", "didn't": "did not", "don't": "do not", "you're": "you are", "you've": "you would have", "you'll": "you will", "you'd": "you would", "we're": "we are", "we've": "we have"
    }
    contractions_pattern = re.compile(r'\b(' + '|'.join(contractions.keys()) + r')\b', re.IGNORECASE)
    def replace(match):
        return contractions[match.group(0).lower()]
    return contractions_pattern.sub(replace, text)


def clean_and_refine_text(text):
    text = re.sub(r'\.{2,}', ': ', text)

    text = correct_split_hyphenated_words(text)
    text = nlp_handle_hyphenated_words(text)
    text = preserve_hyphenated_words(text)
    text = expand_contractions(text)
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'Page \d+ of \d+|\[.*?\]', '', text)
    text = re.sub(r'Copyright|All rights reserved|Confidential', '', text, flags=re.I)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\x00-\x7f]', '', text)
    text = re.sub(r'(\w)([A-Z])', r'\1 \2', text)
    text = re.sub(r'[^\w\s,.%-]', '', text)
    text = re.sub(r'(\n|^)(\d+\s+)?([A-Z][\w\s]+):', r'\n\n\3:\n', text)
    text = re.sub(r'(\d{1,2}) %', r'\1%', text)
    text = re.sub(r'\b(\d+)\b(?!\%)', r' \1 ', text)
    text = re.sub(r'\.([A-Z])', r'. \1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\.\s+', ' ', text)
    text = re.sub(r'\s+(\d+)', r' \1', text)
    
    return text



cleaned_and_refined_texts = [clean_and_refine_text(text) for text in pdf_as_text]

for text in cleaned_and_refined_texts:
    print(text)

Heidelberg your new home for museum and public library included I lost my heart in Heidelberg Name of a 1927 musical by Fred Raymond music and Fritz Lhner Beda and Ernst Neubach words Welcome to Heidelberg Dear new citizen, I am pleased that you have chosen our city as your new home Heidelberg is known for its scenic beauty, its internationality and diversity Living in Heidelberg means having a home the city derives its high quality of life from the attractive mix of living and working, research and culture As a cosmopolitan city and Rainbow City, Heidelberg welcomes all people with open arms, regardless of their origin or sexual identity We want you to feel at home here and to that end, we support you in all areas of life Families with children have access to an excellent childcare network Schoolchildren can develop their talents to the full here, because Heidelberg is the best school location in Germany according to several studies Low income families receive targeted support in the 

4) Data Pre-Processing

In [54]:
#import nltk
#nltk.download('punkt')
#from nltk.tokenize import word_tokenize

In [55]:
#import nltk
#nltk.download('maxent_ne_chunker')

In [56]:
# def tokenize_text(text):
#     return word_tokenize(text)

# from nltk.corpus import stopwords

# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))

# def remove_stop_words(tokens):
#     return [word for word in tokens if word.lower() not in stop_words]

# from nltk.stem import WordNetLemmatizer

# nltk.download('wordnet')

# lemmatizer = WordNetLemmatizer()

# def lemmatize_words(tokens):
#     return [lemmatizer.lemmatize(word) for word in tokens]
# nltk.download('averaged_perceptron_tagger')

# def tag_parts_of_speech(tokens):
#     return nltk.pos_tag(tokens)

# from nltk.chunk import ne_chunk

# def extract_named_entities(tagged_tokens):
#     return ne_chunk(tagged_tokens)

# def normalize_text(tokens):
#     return [token.lower() for token in tokens]

# def preprocess_text(text):
#     tokens = tokenize_text(text)
#     tokens = normalize_text(tokens)
#     tokens = remove_stop_words(tokens)
#     tokens = lemmatize_words(tokens)
#     tagged_tokens = tag_parts_of_speech(tokens)
#     named_entities = extract_named_entities(tagged_tokens)
#     return named_entities


# text = preprocess_text(text)
# print(text)

Using GPT Model

In [2]:
import tensorflow as tf
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModelForSequenceClassification, TextClassificationPipeline
from transformers import Trainer
import torch
import accelerate
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import pipeline

In [58]:
%pip show torch

Name: torch
Version: 2.2.2
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: c:\Users\asha4\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, bitsandbytes, optimum, torchaudio, torchvision
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)

False
2.2.2+cpu


In [4]:
def setup_model(model_name='gpt2'):
    # Load tokenizer and model from the GPT-2 model
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    if torch.cuda.is_available():
        model.cuda()  # Move model to GPU if available
    return tokenizer, model

def generate_response(tokenizer, model, text, max_length=200):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)
    if torch.cuda.is_available():
        input_ids = input_ids.cuda()
        attention_mask = attention_mask.cuda()

    # Generate a response using the model
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        num_beams=5,
        no_repeat_ngram_size=2,
        temperature=0.7,  # Lower temperature
        top_k=50,
        top_p=0.85,  # Adjust top_p to control diversity
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Initialize model and tokenizer
tokenizer, model = setup_model()

# Define a text prompt
text = "Hello, I am your personal chatbot"

# Generate a response from the model
response = generate_response(tokenizer, model, text, max_length=30)
print(response)



c:\Users\asha4\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\asha4\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Hello, I am your personal chatbot.

If you have any questions, please don't hesitate to contact me. I will be happy to


Testing

In [5]:
test_queries = [
    "When is the registration deadline?",
    "How can I apply for financial aid?",
    "What courses are required for a Computer Science major?",
    "Tell me more about campus facilities."
]

responses = []
for query in test_queries:
    response = generate_response(tokenizer, model, query)
    responses.append(response)
    print(f"Query: {query}\n Response: {response}\n")


Query: When is the registration deadline?
 Response: When is the registration deadline?

The deadline for registration is July 1st, 2018. Registration is open to the general public and is subject to change at any time. If you have any questions, please contact us.

Query: How can I apply for financial aid?
 Response: How can I apply for financial aid?

You can apply online or by phone at 1-800-273-TALK (8255) or online at www.gofundme.com.

Query: What courses are required for a Computer Science major?
 Response: What courses are required for a Computer Science major?

Computer Science majors must take at least one of the following courses:

Query: Tell me more about campus facilities.
 Response: Tell me more about campus facilities.

"I think it's important for us to be able to provide a safe environment for all of our students and faculty," he said. "We need to make sure that we have the facilities that are right for our campus, and that's what we're trying to do. We're going to cont

In [6]:
test_queries = [
    "What is the registration deadline for new students?",
    "How can a freshman apply for financial aid?",
    "Which core courses are required for a Computer Science major?",
    "Describe the campus facilities available to all students."
]

responses = []
for query in test_queries:
    response = generate_response(tokenizer, model, query, max_length=100)  # Slightly increase max_length for more detailed responses
    responses.append(response)
    print(f"Query: {query}\n Response: {response}\n")


Query: What is the registration deadline for new students?
 Response: What is the registration deadline for new students?

If you are a new student, you will be required to register at the end of the first year of your degree program. If you do not have a valid student ID card, your registration will not be accepted until you have completed your first semester of college. You will also need to complete an online application to become a registered student in order to be eligible to apply for a certificate of enrollment in the College of Arts and Sciences (CAS).

Query: How can a freshman apply for financial aid?
 Response: How can a freshman apply for financial aid?

If you are applying for a scholarship, you will need to submit a letter of application to the Office of Financial Aid (OFA), which will be sent to you by the end of the year. The letter must include the following information: your name, address, phone number, and the date you applied for the scholarship. If you do not recei

Save the Model as well as the Tokenizer

In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Assuming you have already initialized and potentially trained/fine-tuned your model
model = GPT2LMHeadModel.from_pretrained('gpt2')  # Or your fine-tuned model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def save_model_and_tokenizer(model, tokenizer, save_directory):
    """
    Saves the model and tokenizer to the specified directory.
    """
    import os
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    model.save_pretrained(save_directory)
    tokenizer.save_pretrained(save_directory)

save_directory = 'C:\\Users\\asha4\\OneDrive\\Desktop\\mini\\mini'
save_model_and_tokenizer(model, tokenizer, save_directory)